In [1]:
from Crawling import Crawling

crawl = Crawling()
BGM_data = crawl.get_BGM_data()

START crawling
FIND 19 lists in 날림v CHANNEL
GET BGM Data from 날림v CHANNEL in 58.70s
END crawling


In [2]:
from Database import Database
DB = Database('alskfl')
DB.update_song(BGM_data)
DB.update_list(BGM_data)
DB.update_song_list(BGM_data)

DATABASE READY
USER : alskfl
Crawled BGM count: 1230
Before BGM count: 1230
New BGM count: 0
Deleted BGM count: 0
Table song updated
Crawled list count: 19
Inserted list count: 0
Deleted list count: 0
Table list updated
BGM 2020 07 : 0 songs added
BGM 2020 08 : 0 songs added
BGM 2020 09 : 0 songs added
BGM 2020 10 : 0 songs added
BGM 2020 11 : 0 songs added
BGM 2020 12 : 0 songs added
BGM 2021 01 : 0 songs added
BGM 2021 02 : 0 songs added
BGM 2021 03 : 0 songs added
BGM 2021 04 : 0 songs added
BGM 2021 05 : 0 songs added
BGM 2021 06 : 0 songs added
BGM 2021 07 : 0 songs added
BGM 2021 08 : 0 songs added
BGM 2021 09 : 0 songs added
BGM 2021 10 : 0 songs added
BGM 2021 11 : 0 songs added
BGM 2021 12 : 0 songs added
BGM 2022 01 : 0 songs added


![](youtube_playlist.png)